In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Pipelines:  model upload, predict, and evaluate using google-cloud-pipeline-components

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_model_upload_predict_evaluate.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/notebooks/blob/master/official/pipelines/google_cloud_pipeline_components_model_upload_predict_evaluate.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/notebooks/blob/master/official/pipelines/google_cloud_pipeline_components_model_upload_predict_evaluate.ipynb">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview

This notebook shows how to use the components defined in [`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) in conjunction with an experimental `evaluation` method, to build a [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines) workflow that uploads a tabular custom model as a `Model` resource, creates a `BatchPredictionJob` resource, and evaluates the `Model` resource with the `BatchPredictionJob` results to create an evaluation `system.Metrics` artifact.

### Dataset

The dataset used for this tutorial is part of the [safe driver prediction Kaggle competition](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/overview). The model has been trained on this data, and ground truth will be used for evaluation.

The dataset predicts the whether or not a claim was filed for the policy holder.

### Objective

In this tutorial, you evaluate a custom model using a pipeline with components from `google_cloud_pipeline_components` and a custom pipeline component you build.

The steps performed include:

- Upload a pre-trained model as a `Model` resource.
- Run a `BatchPredictionJob` on the `Model` resource with ground truth data.
- Generate evaluation `Metrics` artifact about the `Model` resource.
- Compare the evaluation metrics to a threshold.


### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

If you are using Colab or Google Cloud Notebook, your environment already meets all the requirements to run this notebook. You can skip this step.

Otherwise, make sure your environment meets this notebook's requirements. You need the following:

- The Cloud Storage SDK
- Git
- Python 3
- virtualenv
- Jupyter notebook running in a virtual environment with Python 3

The Cloud Storage guide to [Setting up a Python development environment](https://cloud.google.com/python/setup) and the [Jupyter installation guide](https://jupyter.org/install) provide detailed instructions for meeting these requirements. The following steps provide a condensed set of instructions:

1. [Install and initialize the SDK](https://cloud.google.com/sdk/docs/).

2. [Install Python 3](https://cloud.google.com/python/setup#installing_python).

3. [Install virtualenv](Ihttps://cloud.google.com/python/setup#installing_and_using_virtualenv) and create a virtual environment that uses Python 3.

4. Activate that environment and run `pip3 install Jupyter` in a terminal shell to install Jupyter.

5. Run `jupyter notebook` on the command line in a terminal shell to launch Jupyter.

6. Open this notebook in the Jupyter Notebook Dashboard.


## Installation

Install the latest version of Vertex AI SDK for Python.

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest GA version of *google-cloud-pipeline-components* library as well.

In [ ]:
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG
! pip3 install --upgrade kfp $USER_FLAG

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Check the versions of the packages you installed.  

In [ ]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

## Before you begin

### GPU runtime

This tutorial does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. [The Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebook.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebook**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

**Click Create service account**.

In the **Service account name** field, enter a name, and click **Create**.

In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex" into the filter box, and select **Vertex Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

Click Create. A JSON file that contains your key downloads to your local environment.

Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex AI SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your GCP project id from gcloud
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
    print("Service Account:", SERVICE_ACCOUNT)

In [ ]:
shell_output[2].replace("*", "").strip()

#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step -- you only need to run these once per service account.

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_NAME

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import google.cloud.aiplatform as aip
from kfp.v2.dsl import Input, Metrics, component

#### Vertex AI Pipelines constants

Setup up the following constant for Vertex AI Pipelines:

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/safe_driver".format(BUCKET_NAME)

## Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME, location=REGION)

## Create component for comparing evalution metrics to a threshold

First, you create your own component that will take as input the evaluation metrics artifact and make a comparison to a threshold and return a yes/no decision that could be used in a subsequent dsl.Condition() to decide whether the model should proceed to the next step -- e.g., online deployment.

The component takes the following parameters:

- `eval_metrics`: The evaluation metrics artifact returned from `ModelEvaluation` component.
- `metric_name`: The key name for the metric entry to make the comparison to.
- `threshold`: The threshold for the metric value for a yes/no decision.

In [ ]:
@component()
def compare(eval_metrics: Input[Metrics], metric_name: str, threshold: float) -> str:
    path = eval_metrics.path
    # print("PATH", path)

    gs_prefix = "gs://"
    gcsfuse_prefix = "/gcs/"
    if path.startswith(gs_prefix):
        path = path.replace(gs_prefix, gcsfuse_prefix)

    import json

    with open(path, "r") as f:
        data = json.load(f)

    slices = data["slicedMetrics"]
    # print("# slices", len(slices))

    metrics = slices[0]["metrics"]["classification"]
    # print("METRIC KEYS", metrics.keys())

    value = metrics[metric_name]
    if value > threshold:
        return "true"

    return "false"

## Define evaluation pipeline that uses components from `google_cloud_pipeline_components`

Next, define the pipeline.

Then, [`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) components are used to define the rest of the pipeline: upload the model, run batch prediction, and evaluate the model with the given predictions.

View the definition of the [upload model component](https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/aiplatform/model/upload_model/component.yaml).

View the definition of the [batch prediction component](https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/aiplatform/batch_predict_job/component.yaml).

View the definition of the [experimental evaluation component](https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/experimental/evaluation/component.yaml).

In [ ]:
import kfp
from google_cloud_pipeline_components.experimental.evaluation import \
    ModelEvaluationOp as evaluation_op
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components.v1.batch_predict_job import \
    ModelBatchPredictOp as batch_prediction_op
from google_cloud_pipeline_components.v1.model import \
    ModelUploadOp as model_upload_op
from kfp.v2.components import importer_node

DATA_URIS = [
    "gs://cloud-samples-data/vertex-ai/dataset-management/datasets/safe_driver/dataset_safe_driver_train_10k.csv"
]
MODEL_URI = "gs://cloud-samples-data/vertex-ai/google-cloud-aiplatform-ci-artifacts/models/safe_driver/model"
# Create working dir
WORKING_DIR = f"{PIPELINE_ROOT}/{TIMESTAMP}"
MODEL_DISPLAY_NAME = f"safe-driver-{TIMESTAMP}"
BATCH_PREDICTION_DISPLAY_NAME = f"batch-prediction-on-pipelines-model-{TIMESTAMP}"


@kfp.dsl.pipeline(name="upload-evaluate-" + TIMESTAMP)
def pipeline(
    metric: str,
    threshold: float,
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    batch_prediction_display_name: str = BATCH_PREDICTION_DISPLAY_NAME,
    batch_prediction_data_uris: list = DATA_URIS,
):
    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=MODEL_URI,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "artifactUri": MODEL_URI,
            "predictSchemata": {
                "predictionSchemaUri": MODEL_URI + "/prediction_schema.yaml",
                "instanceSchemaUri": MODEL_URI + "/instance.yaml",
            },
            "containerSpec": {
                "imageUri": "us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server:prod",
                "healthRoute": "/health",
                "predictRoute": "/predict",
            },
        },
    )

    model_task = model_upload_op(
        project=project,
        display_name=model_display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    )

    batch_prediction_task = batch_prediction_op(
        project=project,
        model=model_task.outputs["model"],
        job_display_name=batch_prediction_display_name,
        gcs_source_uris=batch_prediction_data_uris,
        instances_format="csv",
        predictions_format="jsonl",
        gcs_destination_output_uri_prefix=WORKING_DIR,
        machine_type="n1-standard-32",
    )

    eval_task = evaluation_op(
        project=project,
        root_dir=WORKING_DIR,
        problem_type="classification",
        classification_type="multiclass",
        ground_truth_column="target",
        class_names=["0", "1"],
        predictions_format="jsonl",
        batch_prediction_job=batch_prediction_task.outputs["batchpredictionjob"],
    )

    _ = compare(
        eval_metrics=eval_task.outputs["evaluation_metrics"],
        metric_name=metric,
        threshold=threshold,
    )

## Compile the pipeline

Next, compile the pipeline.

In [ ]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="evaluation_demo_pipeline.json",
)

## Run the pipeline

Next, run the pipeline.

In [ ]:
DISPLAY_NAME = "safe_driver" + TIMESTAMP

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="evaluation_demo_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"metric": "auPrc", "threshold": 0.95},
    enable_caching=True,
)

job.run()

! rm evaluation_demo_pipeline.json

Click on the generated link to see your run in the Cloud Console.

In the UI, many of the pipeline DAG nodes will expand or collapse when you click on them.

### View the pipeline results and evaluation metrics

In [ ]:
PROJECT_NUMBER = job.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    artifact = ""
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        EVALUATION_METRICS = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/evaluation_metrics"
        )
        # Check if file exists, 0 is success
        !gsutil -q stat $EXECUTE_OUTPUT
        if _exit_code == 0:
            ! gsutil cat $EXECUTE_OUTPUT
            artifact = EXECUTE_OUTPUT
            break
        !gsutil -q stat $GCP_RESOURCES
        if _exit_code == 0:
            ! gsutil cat $GCP_RESOURCES
            artifact = GCP_RESOURCES
            break
        !gsutil -q stat $EVALUATION_METRICS
        if _exit_code == 0:
            ! gsutil cat $EVALUATION_METRICS
            artifact = EVALUATION_METRICS
            break

    return artifact


print("model-upload")
artifacts = print_pipeline_output(job, "model-upload")
print("\n\n")
print("model-batch-predict")
artifacts = print_pipeline_output(job, "model-batch-predict")
print("\n\n")
print("model-evaluation")
metrics = print_pipeline_output(job, "model-evaluation")
print("\n\n")
print("compare")
artifacts = print_pipeline_output(job, "compare")

### Delete a pipeline job

After a pipeline job is completed, you can delete the pipeline job with the method `delete()`.  Prior to completion, a pipeline job can be canceled with the method `cancel()`.

In [ ]:
job.delete()

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial -- *Note:* this is auto-generated and not all resources may be applicable for this tutorial:

- Model
- Batch Job
- Cloud Storage Bucket

In [ ]:
delete_model = True
delete_batchjob = True
delete_bucket = True

try:
    if delete_model and "MODEL_DISPLAY_NAME" in globals():
        models = aip.Model.list(
            filter=f"display_name={MODEL_DISPLAY_NAME}", order_by="create_time"
        )
        model = models[0]
        aip.Model.delete(model)
        print("Deleted model:", model)
except Exception as e:
    print(e)

try:
    if delete_batchjob and "BATCH_PREDICTION_DISPLAY_NAME" in globals():
        batch_predictions = aip.BatchPredictionJob.list(
            filter=f"display_name={BATCH_PREDICTION_DISPLAY_NAME}",
            order_by="create_time",
        )
        batch_prediction = batch_predictions[0]
        aip.BatchPredictionJob.delete(batch_prediction)
        print("Deleted batch prediction job:", batch_prediction)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME